# Hamiltonian Monte Carlo

## Resources

* https://towardsdatascience.com/python-hamiltonian-monte-carlo-from-scratch-955dba96a42d
* Simulation-based Probabilistic Risk Assessment : https://arxiv.org/pdf/2207.12575.pdf
* https://zhijingeu.medium.com/building-a-probabilistic-risk-estimate-using-monte-carlo-simulations-with-python-mcerp-7d57e63112fa